In [ ]:
!pip install timm

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import copy
from matplotlib_venn import venn2
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score , f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import KFold , StratifiedKFold, GroupKFold
from tqdm.notebook import tqdm
tqdm.pandas() 
from sklearn.metrics import auc
from gensim.models import word2vec, KeyedVectors

In [ ]:
!cp "/content/drive/MyDrive/mycompetition/atma/#13/atmaCup13_dataset.zip" .
!unzip atmaCup13_dataset.zip

In [ ]:
cart = pd.read_csv("/content/cart_log.csv")
coupon = pd.read_csv("/content/coupon.csv")
session = pd.read_csv("/content/session.csv")
product = pd.read_csv("/content/product_master.csv")
test = pd.read_csv("/content/test_session.csv")
price = pd.read_csv("/content/price.csv")
sample = pd.read_csv("/content/atmaCup13_sample_submission.csv")
coupon_jan = pd.read_csv("/content/coupon_target_jan.csv")

In [ ]:
def make_data():
  df = session
  df["is_train"] = 1
  df.loc[df["session_id"].isin(test["session_id"]), "is_train"] = 0

  jan = product[product["category"] == "チーズ"]
  t = cart[(cart["JAN"].isin(jan.unique()))&(cart["duration"] >= 180)].session_id.unique()
  #data_dfにtargetを付随する
  df["target"] = 0
  df.loc[data_df.session_id.isin(t), "target"] = 1

  return df

data_df = make_data()

In [ ]:
#cart情報からsession_idごとの特徴量を作る

cart["created_at__date"] = pd.to_datetime(cart["created_at__date"])
cart["month"] = cart["created_at__date"].apply(lambda x:x.month)
cart["day"] = cart["created_at__date"].apply(lambda x:x.day)
cart["week-of-day"] = cart["created_at__date"].apply(lambda x:x.dayofweek)

In [ ]:
#priceのデータをつけてみる
price["created_at__date"] = pd.to_datetime(price["date"])
cart = pd.merge(cart,price[["avg_price","min_price","max_price","created_at__date","JAN"]],on=["created_at__date","JAN"],how="left")

cart_df = pd.merge(cart,session[["session_id", "user_id"]],on="session_id",how="left")
#cart_df["diff_price"] = cart_df["max_price"] - cart_df["min_price"]
#180sよりもまえに購入しているもののデータ
cart_df_b = cart_df[cart_df["duration"] < 180].reset_index(drop=True)
cart_df_b

,session_id,duration,action_name,action_detail,item_name,item_detail,item_detail_add_1,item_detail_add_2,coupon_is_activated,JAN,n_items,created_at__date,created_at__hour,month,day,week-of-day,avg_price_x,min_price_x,max_price_x,avg_price_y,min_price_y,max_price_y,avg_price,min_price,max_price,user_id
0,00003ee1b7f34620ae81dba533c6d5a544513004fe5688...,26,PIN入力,PIN入力,キー,****,NaN,NaN,0,NaN,0,2021-12-04,11,12,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5e9b4221190f5cc4d678381fc2ba592426b6ee2e6d4eb2...
1,00011e97d485ffb7f78662972da6032d4103b1628d541f...,9,PIN入力,PIN入力,キー,****,NaN,NaN,0,NaN,0,2021-11-18,19,11,18,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ce397b59fd9633c11a8b659d8ba49ab024cbe9186c2d30...
2,00011e97d485ffb7f78662972da6032d4103b1628d541f...,49,買物,商品追加,商品,4902888219368,ダース ミルク,1.0,0,4.902888e+12,1,2021-11-18,19,11,18,3,69.000000,69.0,69.0,69.000000,69.0,69.0,69.000000,69.0,69.0,ce397b59fd9633c11a8b659d8ba49ab024cbe9186c2d30...
3,00011e97d485ffb7f78662972da6032d4103b1628d541f...,51,買物,商品追加,商品,4902888219368,ダース ミルク,1.0,0,4.902888e+12,1,2021-11-18,19,11,18,3,69.000000,69.0,69.0,69.000000,69.0,69.0,69.000000,69.0,69.0,ce397b59fd9633c11a8b659d8ba49ab024cbe9186c2d30...
4,00011e97d485ffb7f78662972da6032d4103b1628d541f...,81,買物,商品追加,商品,4580030180010,パイン２００ｇ,1.0,0,4.580030e+12,1,2021-11-18,19,11,18,3,199.000000,199.0,199.0,199.000000,199.0,199.0,199.000000,199.0,199.0,ce397b59fd9633c11a8b659d8ba49ab024cbe9186c2d30...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434694,ffff3b41d0dadb1efd1d413aa9e8bbe66349455576c437...,152,買物,商品追加,商品,2544336000000,豚肩ロースしゃぶ用（大）,1.0,0,2.544336e+12,1,2022-01-29,18,1,29,5,584.521739,546.0,631.0,584.521739,546.0,631.0,584.521739,546.0,631.0,93cfdde9d49063b4734e2e28b048ad645edab09e684e72...
434695,ffffdde1709024552278324dcbbeb8bc0e2bb39e462eb0...,18,PIN入力,PIN入力,キー,****,NaN,NaN,0,NaN,0,2021-12-18,9,12,18,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ec06de68c1988c1fcfba8c570db46618ea523b408d8caa...
434696,ffffdde1709024552278324dcbbeb8bc0e2bb39e462eb0...,52,買物,商品追加,商品,4908392160028,鏡もち（小）,1.0,0,4.908392e+12,1,2021-12-18,9,12,18,5,499.000000,499.0,499.0,499.000000,499.0,499.0,499.000000,499.0,499.0,ec06de68c1988c1fcfba8c570db46618ea523b408d8caa...
434697,ffffdde1709024552278324dcbbeb8bc0e2bb39e462eb0...,82,買物,商品追加,商品,4973360720474,サッと鏡餅 まる餅入り,1.0,0,4.973361e+12,1,2021-12-18,9,12,18,5,649.000000,649.0,649.0,649.000000,649.0,649.0,649.000000,649.0,649.0,ec06de68c1988c1fcfba8c570db46618ea523b408d8caa...


In [ ]:
#word2vecで購入したもののベクトル表現を得る
import os
def word2vec_vectorizer(input_df, col, replace=True):
  model = word2vec.Word2Vec(
            sentences=input_df[col].tolist(),
            size=32,
            min_count=1,
            window=5,
            sg=1, # skip-gramモデルを使用するか(0: しない, 1: する)
            hs=1, # 頻出単語をはじくか(0: はじく, 1: はじかない)
            iter=5
        )
  vectors = input_df[col].apply(lambda x: np.mean([model[e] for e in x], axis=0))
    
  return vectors

cart_df_b["count"] = cart_df_b.groupby("session_id")["item_detail_add_1"].transform("count")
cart_df_b.loc[cart_df_b["count"] == 0 , "item_detail_add_1"] = "購入無し"
cart_dff = cart_df_b[cart_df_b["item_detail_add_1"].notnull()].reset_index(drop=True)

sentence_vectors = word2vec_vectorizer(cart_dff,"item_detail_add_1")
sentence_vectors = np.vstack([x for x in sentence_vectors])
sentence_vector_df = pd.DataFrame(sentence_vectors,columns=[f"item_detai_w2v_{i}"for i in range(32)])

cart_dff = pd.concat([cart_dff["session_id"],sentence_vector_df],axis=1)

In [ ]:
#cart情報からsession_idごとの特徴量を作る
cart_session = pd.DataFrame(cart_df_b["session_id"].unique(),columns={"session_id"})
cart_session["item_num"] = cart_df_b.groupby("session_id")["n_items"].transform("cumsum")
cart_session["first_action"] = cart_df_b.groupby("session_id")["duration"].transform("first")
cart_session["created_at__hour"] = cart_df_b.groupby("session_id")["created_at__hour"].transform("first")
cart_session["month"] = cart_df_b.groupby("session_id")["month"].transform("first")
cart_session["day"] = cart_df_b.groupby("session_id")["day"].transform("first")
cart_session["week-of-day"] = cart_df_b.groupby("session_id")["week-of-day"].transform("first")
cart_session["coupon_num"] = cart_df_b.groupby("session_id")["coupon_is_activated"].transform("mean")

cart_session = pd.merge(cart_session, cart_dff.groupby("session_id").mean(), on="session_id")

del cart_dff,sentence_vector_df,sentence_vectors

In [ ]:
cart_df["count"] = cart_df.groupby("user_id")["item_detail_add_1"].transform("count")
cart_df.loc[cart_df["count"] == 0 , "item_detail_add_1"] = "購入無し"
cart_dff = cart_df[cart_df["item_detail_add_1"].notnull()].reset_index(drop=True)

sentence_vectors = word2vec_vectorizer(cart_dff,"item_detail_add_1")
sentence_vectors = np.vstack([x for x in sentence_vectors])
sentence_vector_df = pd.DataFrame(sentence_vectors,columns=[f"all_item_detai_w2v_{i}"for i in range(32)])

cart_dff = pd.concat([cart_dff["user_id"],sentence_vector_df],axis=1)

user_feature = pd.DataFrame(cart_df["user_id"].unique(),columns={"user_id"})
user_feature = pd.merge(user_feature, cart_df.groupby("user_id")["session_id"].nunique(),on="user_id")
user_feature.rename(columns={"session_id","purchace_count"})
user_feature["avg_avg_price"] = cart_df.groupby("user_id")["avg_price"].transform("mean")
user_feature["avg_max_price"] = cart_df.groupby("user_id")["max_price"].transform("mean")
user_feature["avg_min_price"] = cart_df.groupby("user_id")["min_price"].transform("mean")

user_feature = pd.merge(user_feature, cart_dff.groupby("user_id").mean(), on="user_id")

In [ ]:
data_df = pd.merge(data_df,cart_session,on="session_id",how="left")
data_df

,session_id,registor_number,user_id,sex,age,distance_to_the_store,start_at__date,start_at__hour,is_train,target,item_num,first_action,created_at__hour,month,day,week-of-day,item_detai_w2v_0,item_detai_w2v_1,item_detai_w2v_2,item_detai_w2v_3,item_detai_w2v_4,item_detai_w2v_5,item_detai_w2v_6,item_detai_w2v_7,item_detai_w2v_8,item_detai_w2v_9,item_detai_w2v_10,item_detai_w2v_11,item_detai_w2v_12,item_detai_w2v_13,item_detai_w2v_14,item_detai_w2v_15,item_detai_w2v_16,item_detai_w2v_17,item_detai_w2v_18,item_detai_w2v_19,item_detai_w2v_20,item_detai_w2v_21,item_detai_w2v_22,item_detai_w2v_23,item_detai_w2v_24,item_detai_w2v_25,item_detai_w2v_26,item_detai_w2v_27,item_detai_w2v_28,item_detai_w2v_29,item_detai_w2v_30,item_detai_w2v_31
0,00003ee1b7f34620ae81dba533c6d5a544513004fe5688...,1097,5e9b4221190f5cc4d678381fc2ba592426b6ee2e6d4eb2...,女性,60~64,2,2021-12-04,11,1,0,0.0,26.0,11.0,12.0,4.0,5.0,-0.355065,0.213834,-0.711697,0.728247,-0.977821,-1.563369,1.444811,0.286052,-1.670916,1.459855,-0.340377,-0.264137,0.256698,1.091617,1.344592,-0.227277,-1.292819,-0.512283,0.613970,0.603794,-0.475759,-0.474150,0.171487,-0.397123,-0.436192,-0.034602,0.328723,-1.175585,-0.777934,-0.925793,0.020317,-0.100531
1,00011e97d485ffb7f78662972da6032d4103b1628d541f...,1045,ce397b59fd9633c11a8b659d8ba49ab024cbe9186c2d30...,女性,55~59,4,2021-11-18,19,1,0,0.0,9.0,19.0,11.0,18.0,3.0,-0.128036,0.097374,-0.380207,0.066948,-0.167446,-0.318448,0.099241,0.095298,0.341272,0.246181,-0.370800,0.000727,-0.179695,0.101266,0.110270,0.213604,-0.075190,-0.223156,-0.049251,0.365030,-0.108045,-0.213015,0.195446,-0.103315,0.339667,0.094866,-0.179157,-0.207938,-0.126532,-0.003536,-0.309712,0.134600
2,000280d2156d179a597167b269a69e5af556725eeaabae...,1016,8da8267f34ec7ee3f3b800b089b7e27c16fe8cf5a7e36b...,女性,35~39,17,2022-01-25,13,0,0,1.0,9.0,19.0,11.0,18.0,3.0,-0.619068,-0.553808,-0.740431,0.433465,-0.143398,-0.001802,0.220006,-0.184175,0.432295,0.245234,-0.116908,0.027079,-0.057405,-0.108827,-0.202020,0.166040,-0.288673,-0.170322,0.034182,0.100708,0.142730,-0.484211,0.056092,-0.217972,0.063830,0.670771,-0.799377,-0.399330,-0.303401,-0.105174,-0.607099,-0.061180
3,0002a4a49d2230456ce57f9709b7c97e1ee324906f4ed8...,1047,b943a6cfb8e1a2b7c97aa9486d1ca7a8ab16ce40a39a72...,女性,30~34,18,2021-12-20,17,1,0,2.0,9.0,19.0,11.0,18.0,3.0,-0.355065,0.213834,-0.711697,0.728247,-0.977821,-1.563369,1.444811,0.286052,-1.670916,1.459855,-0.340377,-0.264137,0.256698,1.091617,1.344592,-0.227277,-1.292819,-0.512283,0.613970,0.603794,-0.475759,-0.474150,0.171487,-0.397123,-0.436192,-0.034602,0.328723,-1.175585,-0.777934,-0.925793,0.020317,-0.100531
4,000383cb2018d041d4976a0e26d2dc0b73918abd5a30c9...,1081,35d09cb3d4bf8998191ea6b4ed8b8a2d0ff089b0eadc96...,女性,25~29,2,2021-11-11,8,1,0,3.0,9.0,19.0,11.0,18.0,3.0,-0.244349,-0.107272,0.054334,0.558692,-0.019407,-0.382493,-0.013486,0.433491,0.283997,0.663862,-0.523787,0.456593,-0.368574,0.274132,-0.132136,0.095363,0.006232,-0.190067,-0.153797,0.002352,-0.537491,0.122289,0.394232,0.033726,0.372842,0.585965,0.204940,-0.434312,-0.037370,0.133685,-0.055721,-0.370702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109156,fffc962b9f45b0059808e4911a71a88e278afd156116ef...,1026,7214f839ff40ccbc2de894959157c6a6c78b7a6af1a395...,女性,35~39,8,2021-12-06,12,1,1,3.0,4.0,18.0,11.0,26.0,4.0,-0.407863,0.192039,-0.254641,0.080692,-0.281777,-0.175522,0.066040,0.006558,0.699444,0.321302,-0.660645,-0.270522,0.181352,-0.084558,-0.189748,0.429260,0.040280,-0.280617,-0.276388,0.364653,-0.289466,-0.659591,-0.037570,-0.212695,0.413472,0.252585,-0.437683,-0.062121,-0.097741,0.386820,-0.044084,0.142485
109157,fffd70401bffc2d2b7c5980bce4d373d693f6ffa26f0bb...,1086,b41f05753fa4c4d81e57e00cf3169c01f4d6ab9125cbb7...,女性,40~44,1,2021-11-26,20,1,0,4.0,4.0,18.0,11.0,26.0,4.0,-0.160817,0.033011,-0.057539,0.003280,-0.163470,-0.324501,0.132020,-0.168243,

In [ ]:
for col in ["sex","age","distance_to_the_store"]:
  le = LabelEncoder()
  data_df[col] = le.fit_transform(data_df[col])

data_df2 = data_df.drop(["session_id","user_id","start_at__date"],axis=1)

In [ ]:
x_train = data_df2[data_df2.is_train == 1].drop(["target"],axis=1).reset_index(drop=True)
y_train = data_df2[data_df2.is_train == 1][["target"]].reset_index(drop=True)
x_test = data_df2[data_df2.is_train == 0].drop(["target"],axis=1).reset_index(drop=True)

In [ ]:
lgb_params = {
    'objective': 'binary',
    'importance_type': 'gain',
    "metric" : "auc",
    'randm_state': 42,
    'n_jobs': -1,
    'verbose': 1,
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    "num_iterations" : 10000,
    "n_estimator" : 150,
    "max_depth":7,
    "num_leaves":80
}

In [ ]:
from sklearn.metrics import roc_auc_score
scores = []
y_preds = []
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(x_train, y_train)):
    x_tr_fold = x_train.iloc[tr_idx]
    y_tr_fold = y_train.iloc[tr_idx]
    x_vl_fold = x_train.iloc[vl_idx]
    y_vl_fold = y_train.iloc[vl_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        x_tr_fold, y_tr_fold,
        eval_set=(x_vl_fold, y_vl_fold),
        eval_metric='auc',
        verbose=500,
        early_stopping_rounds=50,
    )

    y_preds.append(model.predict(x_test))
    score = roc_auc_score(y_vl_fold,model.predict(x_vl_fold))
    scores.append(score)
    print(
        f'fold {fold} score:',score
    )

print('oof score:',np.mean(scores))

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[65]	valid_0's auc: 0.58406
fold 0 score: 0.5840596590988965
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[259]	valid_0's auc: 0.594772
fold 1 score: 0.5947721512937133
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[293]	valid_0's auc: 0.587124
fold 2 score: 0.5871239472713661
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[146]	valid_0's auc: 0.589099
fold 3 score: 0.5890991327672276
Training until validation scores don't improve for 50 rounds.
[500]	valid_0's auc: 0.594141
Early stopping, best iteration is:
[531]	valid_0's auc: 0.594692
fold 4 score: 0.5946921986145862
oof score: 0.589949417809158


In [ ]:
sample["target"] = np.mean(y_preds,axis=0)
sample.to_csv("/content/drive/MyDrive/mycompetition/atma/#13/sub1.csv" , index=False)